In [2]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline


In [3]:
words=open("names.txt",'r').read().splitlines()
words[:10]

['emma',
 'olivia',
 'ava',
 'isabella',
 'sophia',
 'charlotte',
 'mia',
 'amelia',
 'harper',
 'evelyn']

In [5]:
len(words)

32033

In [4]:
chars = sorted(list(set(''.join(words))))
stoi ={s:i for i,s in enumerate(chars)}
stoi['.']=0
itos = {i:s for s, i in stoi.items()}


In [ ]:
#build the dataset
block_size = 3
X,Y = [],[]
for w in words[:5]:
    context = [0] * block_size  #Before reading a word, the context is initialized to [0, 0, 0].
    for ch in w + '.':
        ix = stoi[ch]
        X.append(context)
        Y.append(ix)
        print(''.join(itos[i] for i in context), '->', itos[ix])
        context =  context[1:] + [ix]   #Drops the oldest character (the first one) and appends the new character.

print(X)
print(Y)
X = torch.tensor(X)
Y = torch.tensor(Y)
print(X.shape)
print(Y.shape)



In [38]:
C = torch.randn(27,2)
C[torch.tensor([5,6,7])]

tensor([[ 0.5626,  0.1221],
        [ 0.9569, -0.1832],
        [-2.3517, -0.1857]])

In [39]:
emb = C[X]
emb.shape

torch.Size([32, 3, 2])

In [43]:
w1 = torch.randn(6, 100)
b1 = torch.randn(100)
h = torch.tanh(emb.view(32,6) @ w1 + b1)
h 


tensor([[ 0.3210,  0.1182, -1.0000,  ..., -0.9279,  0.9315, -0.9999],
        [ 0.9995,  0.9959, -0.9998,  ..., -0.9881,  0.9987, -0.4323],
        [ 0.9927,  0.8649, -0.9955,  ..., -0.2787,  0.9761,  0.8147],
        ...,
        [-0.9923,  0.1651, -0.9893,  ..., -0.9425, -0.6034, -0.9998],
        [ 0.9326,  0.8327, -0.9998,  ..., -0.9969,  0.9774, -0.9999],
        [ 0.6487, -0.7011, -1.0000,  ..., -0.9187,  0.9508, -1.0000]])

In [44]:
h.shape

torch.Size([32, 100])

In [47]:
w2= torch.randn((100,27))
b2= torch.randn(27)
logits = h @ w2 + b2
counts = logits.exp()
probs = counts / counts.sum(1, keepdim=True)
loss = -probs[torch.arange(32), Y].log().mean()
loss



tensor(18.9284)

In [52]:
g = torch.Generator().manual_seed(2147483647)
c = torch.randn((27,2), generator=g)
w1= torch.randn((6,100), generator=g)
b1= torch.randn(100, generator=g)
w2= torch.randn((100,27), generator=g)
b2= torch.randn(27, generator=g)
parameters = [c, w1, b1, w2, b2]
sum(p.nelement() for p in parameters)


3481

In [62]:
for p in parameters:
    p.requires_grad = True

In [ ]:
for i in range(1000):
    #forward pass
    emb = c[X]
    h = torch.tanh(emb.view(-1, 6) @ w1 + b1) #(32,100)
    logits = h @ w2 + b2
    loss = F.cross_entropy(logits, Y)
    
    #print loss periodically
    if i % 100 == 0:
        print(f'iteration {i}: loss = {loss.item():.4f}')

    #backward pass
    for p in parameters:
        p.grad = None
    loss.backward()

    #update
    for p in parameters:
        p.data += -0.1 * p.grad

print(f'Final loss: {loss.item():.4f}')



16.910722732543945


TypeError: unsupported operand type(s) for *: 'float' and 'NoneType'